In [19]:
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model
from random import sample, random
import numpy as np
import matplotlib.pyplot as plt

import sys
import os
import time
import re
import json

In [20]:
def get_filepaths(dir:str):
    files=[]
    for filename in os.listdir(dir):
        files.append(os.path.join(dir, filename))
    return files

In [21]:
def generator(filepaths:list, batch_size:int):

    samples = sample(filepaths, batch_size)
    return_x_arr = []
    return_y_arr = []
    for smpl in samples:
        arrs = np.load(smpl)
        return_x_arr.append(np.expand_dims(arrs[0], axis = -1))
        return_y_arr.append(np.expand_dims(arrs[1], axis = -1))
        
    return_x_arr = np.array(return_x_arr)
    return_y_arr = np.array(return_y_arr)
    
    yield return_x_arr, return_y_arr

In [22]:
def conv_block(x,num_filters):
  x=L.Conv2D(num_filters,3, padding="same")(x)
  x=L.BatchNormalization()(x)
  x=L.Activation("relu")(x)

  x=L.Conv2D(num_filters,3, padding="same")(x)
  x=L.BatchNormalization()(x)
  x=L.Activation("relu")(x)

  x=L.Conv2D(num_filters,3, padding="same")(x)
  x=L.BatchNormalization()(x)
  x=L.Activation("relu")(x)

  x=L.Conv2D(num_filters, 3, padding="same")(x)
  x=L.BatchNormalization()(x)
  x=L.Activation("relu")(x)
  return x

In [23]:
def encoder_block(x, num_filters):
  X=conv_block(x, num_filters)
  p=L.MaxPool2D((2,2))(X)
  return X, p

In [24]:
def attention_gate(g, s, num_filters):
  Wg=L.Conv2D(num_filters, 1, padding="same")(s)
  Wg=L.BatchNormalization()(Wg)

  Ws=L.Conv2D(num_filters, 1, padding="same")(s)
  Ws= L.BatchNormalization()(Ws)

  out=L.Activation("relu")(Wg+Ws)
  out=L.Conv2D(num_filters, 1, padding="same")(out)
  out=L.Activation("sigmoid")(out)

  return out*s

In [25]:
def decoder_block(x,s,num_filters):
  x=L.UpSampling2D(interpolation="bilinear")(x)
  s= attention_gate(x,s,num_filters)
  x = L.Concatenate()([x,s])
  x=conv_block(x,num_filters)
  return x

In [26]:
def attention_unet(input_shape):
    
    input_=L.Input(input_shape)
    

    s1, p1 = encoder_block(input_,64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3= encoder_block(p2,256)
    s4, p4= encoder_block(p3, 256)

    b1=conv_block(p4,512)

    d1=decoder_block(b1,s4, 256)
    d2=decoder_block(d1,s3,256)
    d3=decoder_block(d2,s2, 128)
    d4=decoder_block(d3,s1, 64)
    

    outputs=L.Conv2D(1,1,padding="same", activation="relu")(d4)

    model=Model(input_, outputs, name="Attention-Unet")
    return model


In [27]:
if __name__=="__main__":
  input_shape=(512,512,1)
  model= attention_unet(input_shape)
  model.summary()

Model: "Attention-Unet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 512, 512, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_49 (Conv2D)             (None, 512, 512, 64  640         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_44 (BatchN  (None, 512, 512, 64  256        ['conv2d_49[0][0]']              
 ormalization)                  )                                                    

                                                                                                  
 conv2d_57 (Conv2D)             (None, 128, 128, 25  295168      ['max_pooling2d_5[0][0]']        
                                6)                                                                
                                                                                                  
 batch_normalization_52 (BatchN  (None, 128, 128, 25  1024       ['conv2d_57[0][0]']              
 ormalization)                  6)                                                                
                                                                                                  
 activation_52 (Activation)     (None, 128, 128, 25  0           ['batch_normalization_52[0][0]'] 
                                6)                                                                
                                                                                                  
 conv2d_58

 conv2d_67 (Conv2D)             (None, 32, 32, 512)  2359808     ['activation_61[0][0]']          
                                                                                                  
 conv2d_69 (Conv2D)             (None, 64, 64, 256)  65792       ['activation_59[0][0]']          
                                                                                                  
 conv2d_70 (Conv2D)             (None, 64, 64, 256)  65792       ['activation_59[0][0]']          
                                                                                                  
 batch_normalization_62 (BatchN  (None, 32, 32, 512)  2048       ['conv2d_67[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_64 (BatchN  (None, 64, 64, 256)  1024       ['conv2d_69[0][0]']              
 ormalizat

 ormalization)                                                                                    
                                                                                                  
 conv2d_78 (Conv2D)             (None, 128, 128, 25  65792       ['activation_70[0][0]']          
                                6)                                                                
                                                                                                  
 activation_69 (Activation)     (None, 64, 64, 256)  0           ['batch_normalization_69[0][0]'] 
                                                                                                  
 activation_71 (Activation)     (None, 128, 128, 25  0           ['conv2d_78[0][0]']              
                                6)                                                                
                                                                                                  
 up_sampli

 )                              8)                                'activation_51[0][0]']          
                                                                                                  
 concatenate_6 (Concatenate)    (None, 256, 256, 38  0           ['up_sampling2d_6[0][0]',        
                                4)                                'tf.math.multiply_6[0][0]']     
                                                                                                  
 conv2d_86 (Conv2D)             (None, 256, 256, 12  442496      ['concatenate_6[0][0]']          
                                8)                                                                
                                                                                                  
 batch_normalization_78 (BatchN  (None, 256, 256, 12  512        ['conv2d_86[0][0]']              
 ormalization)                  8)                                                                
          

 conv2d_94 (Conv2D)             (None, 512, 512, 64  36928       ['activation_84[0][0]']          
                                )                                                                 
                                                                                                  
 batch_normalization_85 (BatchN  (None, 512, 512, 64  256        ['conv2d_94[0][0]']              
 ormalization)                  )                                                                 
                                                                                                  
 activation_85 (Activation)     (None, 512, 512, 64  0           ['batch_normalization_85[0][0]'] 
                                )                                                                 
                                                                                                  
 conv2d_95 (Conv2D)             (None, 512, 512, 64  36928       ['activation_85[0][0]']          
          

In [28]:
filepaths = get_filepaths("E:\\Yad_İşlenmiş\\")
filepaths

['E:\\Yad_İşlenmiş\\hst_10082_jr_wfpc2_pc_f606w_u8uojr_drz.fitssec0.npy',
 'E:\\Yad_İşlenmiş\\hst_10082_jr_wfpc2_pc_f606w_u8uojr_drz.fitssec1.npy',
 'E:\\Yad_İşlenmiş\\hst_10082_jr_wfpc2_pc_f606w_u8uojr_drz.fitssec10.npy',
 'E:\\Yad_İşlenmiş\\hst_10082_jr_wfpc2_pc_f606w_u8uojr_drz.fitssec11.npy',
 'E:\\Yad_İşlenmiş\\hst_10082_jr_wfpc2_pc_f606w_u8uojr_drz.fitssec12.npy',
 'E:\\Yad_İşlenmiş\\hst_10082_jr_wfpc2_pc_f606w_u8uojr_drz.fitssec13.npy',
 'E:\\Yad_İşlenmiş\\hst_10082_jr_wfpc2_pc_f606w_u8uojr_drz.fitssec14.npy',
 'E:\\Yad_İşlenmiş\\hst_10082_jr_wfpc2_pc_f606w_u8uojr_drz.fitssec15.npy',
 'E:\\Yad_İşlenmiş\\hst_10082_jr_wfpc2_pc_f606w_u8uojr_drz.fitssec16.npy',
 'E:\\Yad_İşlenmiş\\hst_10082_jr_wfpc2_pc_f606w_u8uojr_drz.fitssec17.npy',
 'E:\\Yad_İşlenmiş\\hst_10082_jr_wfpc2_pc_f606w_u8uojr_drz.fitssec18.npy',
 'E:\\Yad_İşlenmiş\\hst_10082_jr_wfpc2_pc_f606w_u8uojr_drz.fitssec19.npy',
 'E:\\Yad_İşlenmiş\\hst_10082_jr_wfpc2_pc_f606w_u8uojr_drz.fitssec2.npy',
 'E:\\Yad_İşlenmiş\\hst_1008

In [29]:
gen = generator(filepaths, 16)

In [30]:
model.compile(optimizer="adam", loss = "mse", metrics=["mse"])

In [31]:
aaa = model.fit(gen, epochs = 2, verbose="auto", batch_size = 1)

Epoch 1/2


KeyboardInterrupt: 

In [23]:
a = generator(filepaths, 16)

In [24]:
a.shape

(16, 512, 512)

In [ ]:
tf.config.list_physical_devices()